In [133]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import lxml
import pandas as pd
import requests
import yes24_dbio as ydb

In [134]:
# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

In [135]:
data2 = pd.read_csv(f"./yes24/new_yes24bestseller20081_202312_1000_v2.csv")
urls = data2['detail_link']
urls = list(set(urls))
len(urls)

C:\Users\haram4th\AppData\Local\Temp\ipykernel_18936\4143412353.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(f"./yes24/new_yes24bestseller20081_202312_1000_v2.csv")


42780

In [136]:
driver.get(urls[0])
driver.find_element(By.CSS_SELECTOR, "#txtOneCommentCount").text.strip("()건")  # 한줄평 총 갯수

'54'

In [125]:
# driver.find_element(By.CSS_SELECTOR, "#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(3) > a").click()

In [ ]:
#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(5)

In [140]:
for d_url in tqdm(urls):
    book_id = d_url.split("/")[-1] # 책 id
    driver.get(d_url)
    driver.find_element(By.CSS_SELECTOR, "#txtOneCommentCount").text.strip("()건")  # 한줄평 총 갯수
    
    driver.find_element(By.CSS_SELECTOR, "#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_reviewCount > a").click()
#     driver.find_element(By.CSS_SELECTOR, "#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(3) > a").click()
    try:
        driver.find_element(By.CSS_SELECTOR, "#commit_recent").click()
        driver.find_element(By.CSS_SELECTOR, ".bgYUI.first.dim").click()
    except Exception as e :
        pass
    soup = bs(driver.page_source, 'lxml')
    # 한줄평 갯수
    n_one_line_review = int(soup.select_one("#txtOneCommentCount").text.strip("()건"))  # 한줄평 총 갯수

    total_one_line_reviews = []

    if n_one_line_review == 0:
        pass
    elif n_one_line_review > 0:
        n_page_one_line_review = int(n_one_line_review) // 6 + 1  # 한줄평 총 페이지 수
        print(f"한줄평 총 갯수: {n_one_line_review}, 한줄평 총 페이지 수{n_page_one_line_review}")
        if n_page_one_line_review == 1:
            total_one_line_reviews += [item.text for item in soup.select("div.cmt_cont > span.txt")]
        elif n_page_one_line_review > 1:
            driver.find_element(By.CSS_SELECTOR, "#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(4)").click() 
            soup2 = bs(driver.page_source, 'lxml')
            total_one_line_reviews += [item.text for item in soup.select("div.cmt_cont > span.txt")]
            page_num_link = soup2.select_one("#infoset_oneCommentList > div:nth-child(2) > div.rvCmt_sortLft > div > a:nth-child(4)")['href'][:-1]
            for p in range(2, n_page_one_line_review+1):
                r = requests.get(f"https://www.yes24.com/{page_num_link}{p}")
                time.sleep(5)
                soup3 = bs(r.text, 'lxml')
                total_one_line_reviews += [item.text for item in soup3.select("div.cmt_cont > span.txt")]
    line_review_result = pd.DataFrame({"book_id" : book_id, "one_line_review" : total_one_line_reviews})
    line_review_result.to_csv(f"./yes24/one_line_reviews/{book_id}_one_line_reviews.csv", index=False)
    ydb.to_mysql(line_review_result, "yes24best", "yes24_one_line_reviews")

  0%|                                                                                        | 0/42780 [00:02<?, ?it/s]

한줄평 총 갯수: 54, 한줄평 총 페이지 수10


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(4)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x009372A3+45731]
	(No symbol) [0x008C2D51]
	(No symbol) [0x007B880D]
	(No symbol) [0x007EB940]
	(No symbol) [0x007EBE0B]
	(No symbol) [0x0081D1F2]
	(No symbol) [0x00808024]
	(No symbol) [0x0081B7A2]
	(No symbol) [0x00807DD6]
	(No symbol) [0x007E31F6]
	(No symbol) [0x007E439D]
	GetHandleVerifier [0x00C40716+3229462]
	GetHandleVerifier [0x00C884C8+3523784]
	GetHandleVerifier [0x00C8214C+3498316]
	GetHandleVerifier [0x009C1680+611968]
	(No symbol) [0x008CCCCC]
	(No symbol) [0x008C8DF8]
	(No symbol) [0x008C8F1D]
	(No symbol) [0x008BB2C7]
	BaseThreadInitThunk [0x76437

In [138]:
soup

<html lang="ko"><head><link href="https://m.yes24.com/Goods/Detail/107990998" rel="canonical"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
<meta content="86400" http-equiv="Accept-CH-Lifetime"/>
<meta content="unsafe-url" name="referrer"/>
<meta content="width=1170" name="viewport"/>
<title>초등수학 심화 공부법 - 예스24 </title>
<meta content="초등수학 심화 공부법" name="title"/>
<meta content="류승재 저" name="author"/>
<meta content="평범한 아이를 고등수학 1등급 만드는 결정적인 힘. 류승재 저. 블루무스. 판매가 19,800원(10% 할인). 포인트 1,100원(5% 적립). 많은 아이들이 할 만한 문제만 풀다가 고등수학 때 무너진다!심화는 선택이 아닌 필수!2020년 1월, 《수학 잘하..." name="description"/>
<meta content="교사의자녀교육법,유튜버의책,우리아이학습멘토,초등수학,평범한 아이를 고등수학 1등급 만드는 결정적인 힘, 류승재 저, 블루무스,1191426351,9791191426359" name="keywords"/>
<!-- Facebook Interface -->
<meta content="초등수학 심화 공

In [139]:
driver.find_element(By.CSS_SELECTOR, "#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(4)").click() 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(4)"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x009372A3+45731]
	(No symbol) [0x008C2D51]
	(No symbol) [0x007B880D]
	(No symbol) [0x007EB940]
	(No symbol) [0x007EBE0B]
	(No symbol) [0x0081D1F2]
	(No symbol) [0x00808024]
	(No symbol) [0x0081B7A2]
	(No symbol) [0x00807DD6]
	(No symbol) [0x007E31F6]
	(No symbol) [0x007E439D]
	GetHandleVerifier [0x00C40716+3229462]
	GetHandleVerifier [0x00C884C8+3523784]
	GetHandleVerifier [0x00C8214C+3498316]
	GetHandleVerifier [0x009C1680+611968]
	(No symbol) [0x008CCCCC]
	(No symbol) [0x008C8DF8]
	(No symbol) [0x008C8F1D]
	(No symbol) [0x008BB2C7]
	BaseThreadInitThunk [0x76437

In [113]:
#infoset_oneCommentList > div:nth-child(2) > div.rvCmt_sortLft > div > a:nth-child(4)

['자녀 수학교육에 도움이 되었엉ㅎ',
 '자세히 써있어서 좋았습니다.',
 '집에서 수학심화하는 부모에게 빛 같은 책',
 '어렵게만 느꼈던 심화에 도전하게 해준 책! ',
 '수학공부방법을 잡게 해준책',
 '추천합니다 ']

In [119]:
#infoset_oneCommentList > div:nth-child(2) > div.rvCmt_sortLft > div > a:nth-child(4)

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0);" class="bgYUI first dim">...</a> is not clickable at point (95, 13). Other element would receive the click: <h2 class="gd_name">...</h2>
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x009372A3+45731]
	(No symbol) [0x008C2D51]
	(No symbol) [0x007B880D]
	(No symbol) [0x007F07AB]
	(No symbol) [0x007EF2AF]
	(No symbol) [0x007ED89B]
	(No symbol) [0x007ECBA1]
	(No symbol) [0x007E4FC0]
	(No symbol) [0x00807FDC]
	(No symbol) [0x007E4A4E]
	(No symbol) [0x00808254]
	(No symbol) [0x0081B7A2]
	(No symbol) [0x00807DD6]
	(No symbol) [0x007E31F6]
	(No symbol) [0x007E439D]
	GetHandleVerifier [0x00C40716+3229462]
	GetHandleVerifier [0x00C884C8+3523784]
	GetHandleVerifier [0x00C8214C+3498316]
	GetHandleVerifier [0x009C1680+611968]
	(No symbol) [0x008CCCCC]
	(No symbol) [0x008C8DF8]
	(No symbol) [0x008C8F1D]
	(No symbol) [0x008BB2C7]
	BaseThreadInitThunk [0x76437BA9+25]
	R

In [107]:
len(total_one_line_reviews)

60